In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@7b53a54f

In [2]:
import spark.implicits._
case class Activity(machine_id: Int, process_id: Int, activity_type: String, timestamp: Double)

val data = Seq[(Int, Int, String, Double)]((0, 0, "start", 0.712), (0, 0, "end", 1.52), (0, 1, "start", 3.14), (0, 1, "end", 4.12), (1, 0, "start", 0.55), (1, 0, "end", 1.55), (1, 1, "start", 0.43), (1, 1, "end", 1.42), (2, 0, "start", 4.1), (2, 0, "end", 4.512), (2, 1, "start", 2.5), (2, 1, "end", 5))
val activity = data.toDF("machine_id", "process_id", "activity_type", "timestamp")
    .as[Activity]
activity.show()


+----------+----------+-------------+---------+
|machine_id|process_id|activity_type|timestamp|
+----------+----------+-------------+---------+
|         0|         0|        start|    0.712|
|         0|         0|          end|     1.52|
|         0|         1|        start|     3.14|
|         0|         1|          end|     4.12|
|         1|         0|        start|     0.55|
|         1|         0|          end|     1.55|
|         1|         1|        start|     0.43|
|         1|         1|          end|     1.42|
|         2|         0|        start|      4.1|
|         2|         0|          end|    4.512|
|         2|         1|        start|      2.5|
|         2|         1|          end|      5.0|
+----------+----------+-------------+---------+



import spark.implicits._

defined class Activity
data: Seq[(Int, Int, String, Double)] = List(
  (0, 0, "start", 0.712),
  (0, 0, "end", 1.52),
  (0, 1, "start", 3.14),
  (0, 1, "end", 4.12),
  (1, 0, "start", 0.55),
  (1, 0, "end", 1.55),
  (1, 1, "start", 0.43),
  (1, 1, "end", 1.42),
  (2, 0, "start", 4.1),
  (2, 0, "end", 4.512),
  (2, 1, "start", 2.5),
  (2, 1, "end", 5.0)
)
activity: Dataset[Activity] = [machine_id: int, process_id: int ... 2 more fields]

There is a factory website that has several machines each running the same number of processes. Write a solution to find the average time each machine takes to complete a process.

The time to complete a process is the 'end' timestamp minus the 'start' timestamp. The average time is calculated by the total time to complete every process on the machine divided by the number of processes that were run.

The resulting table should have the machine_id along with the average time as processing_time, which should be rounded to 3 decimal places.

Return the result table in any order.

The result format is in the following example.

| machine_id | processing_time |
|------------|-----------------|
| 0          | 0.894           |
| 1          | 0.995           |
| 2          | 1.456           |

In [3]:
case class Machines(machine_id: Int, processing_time: Double)
val processingTimes = activity.groupByKey(a => (a.machine_id, a.process_id))
    .mapGroups{
        (grouped: (Int, Int), iter: Iterator[Activity]) => 
        val timestamps = iter.map(_.timestamp).toList
        val processing_time = timestamps.max - timestamps.min
        Machines(grouped._1, processing_time)
    }
processingTimes.show()


+----------+------------------+
|machine_id|   processing_time|
+----------+------------------+
|         0|             0.808|
|         0|              0.98|
|         1|               1.0|
|         1|              0.99|
|         2|0.4119999999999999|
|         2|               2.5|
+----------+------------------+



defined class Machines
processingTimes: Dataset[Machines] = [machine_id: int, processing_time: double]

In [4]:
processingTimes.groupByKey(a => a.machine_id)
    .mapGroups{
        (machine_id, iter: Iterator[Machines]) =>
        val avg = (math rint iter.map(_.processing_time).sum * 1000)/1000
        Machines(machine_id, avg)
    }.show

+----------+---------------+
|machine_id|processing_time|
+----------+---------------+
|         0|          1.788|
|         1|           1.99|
|         2|          2.912|
+----------+---------------+



In [5]:
activity.groupBy("machine_id", "process_id")
    .agg(
        min("timestamp").as("ts_min"),
        max("timestamp").as("ts_max")
    ).select(col("machine_id"), (col("ts_max") - col("ts_min")).as("processing_time"))
    .groupBy("machine_id")
    .agg(round(avg("processing_time"), 3).as("processing_time"))
    .show()

+----------+---------------+
|machine_id|processing_time|
+----------+---------------+
|         1|          0.995|
|         2|          1.456|
|         0|          0.894|
+----------+---------------+



In [6]:
spark.stop()